# **Курсовая работа**
по теме "Интерпретируемые методы выделения сообществ на атрибутированных графовых наборах данных"\
студент 3 курса, учебной группы ПМ21-4\
факультет ИТиАБД\
Тумасян Валентин Максимович

## **Описание работы**

1. **Наименование темы:** \
Интерпретируемые методы выделения сообществ на атрибутированных графовых наборах данных.

2. **Цель:**\
Изучить и сравнить различные интерпретируемые методы выделения сообществ на атрибутированных графовых наборах данных, а также разработать эффективные алгоритмы их применения на практике.

3. **Задачи:**
* Обзор литературы и существующих методов выделения сообществ (изучение основных концепций, обзор методов)
* Подготовка данных (выбрать атрибутированные графовые датасеты для экспериментов, предварительный анализ данных)
* Реализация выбранных интерпретируемых методов выделения сообществ (кодом на Python)
* Проведение экспериментов (применение реализованных методов, сравнение эффективности и интерпретируемости различных алгоритмов)
* Анализ результатов экспериментов (выявление особенностей каждого метода, сравнение полученных результатов)
* Оценка применимости методов в реальных задачах (протестировать наилучшие методы на реальных задачах, при возможности)
* Составить заключение всей работы

4. **Ожидаемые результаты:**
* Сравнительный анализ интерпретируемых методов выделения сообществ на атрибутированных графовых наборах данных
* Разработанные алгоритмы и код на Python для применения выбранных методов
* Выявление особенностей и преимуществ конкретных методов в различных контекстах
* Рекомендации по выбору наиболее подходящих методов для конкретных прикладных задач

5. **Описание датасета:**\
Репозиторий содержит несколько (3) наборов атрибутивных графовых данных с основными классами истинности. Эти наборы данных были предварительно обработаны и очищены. Наборы данных содержат вершины, ребра и атрибуты узлов, представляющие собой пользователей тех или иных социальных сетей (facebook*, google plus, twitter), дружбу между этими пользователями и профили этих пользователей соответственно.

[*Источник датасета*](https://github.com/he-tiantian/Attributed-Graph-Data): *T. He, L. Bai and Y.S. Ong, "Vicinal Vertex Allocation for Matrix Factorization in Networks," IEEE Transactions on Cybernetics, 2021.*

## **Практическая часть**

### Загрузка наборов данных

In [ ]:
import zipfile
import os

In [ ]:
# путь к файлу архива
zip_file_path = '/content/Attributed-Graph-Data-master.zip'

# распаковываем архив
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
# распаковка подархива и перемещение его содержимого в указанную папку
def extract_dataset(dataset_name, output_folder):
    zip_file_path = f'/content/{dataset_name}.zip'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder)

# создаем папки для каждого датасета и распаковываем содержимое
datasets = ['facebook', 'googleplus', 'twitter']
for dataset in datasets:
    dataset_folder = f'/content/{dataset}'
    os.makedirs(dataset_folder, exist_ok=True)
    extract_dataset(dataset, dataset_folder)

### Создание графов

In [ ]:
import networkx as nx

In [ ]:
folder_paths = ['facebook', 'googleplus', 'twitter']

# обрабатываем данные из каждой папки
for folder_path in folder_paths:
    # создаем пустой граф
    G = nx.Graph()
    # считываем данные из файлов и добавляем их в граф
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # пропускаем папки и ненужные файлы
        if os.path.isdir(file_path) or file_name.startswith('.'):
            continue
        # определяем тип данных по названию файла и загружаем соответствующим образом
        if file_name == 'edgelist':
            # загружаем список рёбер
            with open(file_path, 'r') as f:
                for line in f:
                    source, target = line.strip().split()
                    G.add_edge(source, target)
        elif file_name == 'vertex2aid':
            # загружаем атрибуты вершин
            with open(file_path, 'r') as f:
                for line in f:
                    vertex, attribute = line.strip().split()
                    # проверяем, существует ли вершина в графе
                    if not G.has_node(vertex):
                        G.add_node(vertex)
                    # добавляем атрибут к вершине
                    G.nodes[vertex]['attribute'] = attribute
    # присваиваем соответствующим переменным
    if folder_path == 'facebook':
        G_facebook = G
    elif folder_path == 'googleplus':
        G_googleplus = G
    elif folder_path == 'twitter':
        G_twitter = G

In [ ]:
# выводим информацию о каждом атрибутированном графе
for graph_name, graph in [('G_facebook', G_facebook), ('G_googleplus', G_googleplus), ('G_twitter', G_twitter)]:
    print(f"Граф '{graph_name}':")
    print(f"Количество вершин: {graph.number_of_nodes()}")
    print(f"Количество рёбер: {graph.number_of_edges()}")
    print()

Граф 'G_facebook':
Количество вершин: 4039
Количество рёбер: 88234

Граф 'G_googleplus':
Количество вершин: 7805
Количество рёбер: 321268

Граф 'G_twitter':
Количество вершин: 2455
Количество рёбер: 37154



### Реализация интерпретируемых методов выделения сообществ

##### 1. KMeans

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
# выделение сообществ методом KMeans
def kmeans_community_detection(graph, num_communities):
    # получаем атрибуты вершин в виде векторов одинаковой длины
    node_attributes = {node: np.array(list(data.values())) for node, data in graph.nodes(data=True)}
    attribute_length = len(next(iter(node_attributes.values())))
    for node, attributes in node_attributes.items():
        if len(attributes) < attribute_length:
            node_attributes[node] = np.concatenate([attributes, np.zeros(attribute_length - len(attributes))])

    # применяем KMeans
    kmeans = KMeans(n_clusters=num_communities, random_state=42)
    labels = kmeans.fit_predict(list(node_attributes.values()))

    # формируем словарь, где ключами являются метки кластеров, а значениями - списки вершин в каждом кластере
    communities = {}
    for node, label in zip(graph.nodes(), labels):
        if label not in communities:
            communities[label] = []
        communities[label].append(node)

    return communities

In [ ]:
# пример использования для графа G_facebook с 5 сообществами
num_communities = 5
facebook_communities = kmeans_community_detection(G_facebook, num_communities)
print("сообщества, выделенные методом KMeans, для графа G_facebook:")
for label, nodes in facebook_communities.items():
    print(f"сообщество {label + 1}: {nodes}")

сообщества, выделенные методом KMeans, для графа G_facebook:
сообщество 3: ['0', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '14', '17', '19', '20', '22', '24', '25', '26', '27', '31', '33', '34', '35', '38', '39', '40', '41', '42', '44', '46', '50', '54', '55', '57', '58', '63', '64', '65', '67', '69', '70', '71', '72', '73', '74', '75', '76', '79', '82', '85', '87', '89', '91', '92', '93', '95', '97', '98', '100', '101', '102', '103', '105', '106', '107', '108', '109', '112', '113', '115', '118', '119', '120', '121', '123', '124', '125', '127', '130', '132', '134', '136', '137', '138', '139', '141', '143', '145', '146', '151', '152', '153', '157', '159', '160', '164', '165', '166', '169', '174', '175', '176', '180', '181', '184', '185', '187', '188', '189', '191', '196', '197', '199', '202', '206', '209', '212', '214', '215', '218', '219', '220', '221', '226', '230', '241', '242', '244', '247', '248', '249', '255', '259', '260', '261', '262', '266', '267', '268', '271', '274'

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


##### 2. Spectral Clustering

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
# выделение сообществ методом Spectral Clustering
def spectral_clustering_community_detection(graph, num_communities):
    # получаем матрицу смежности графа в виде массива numpy
    adjacency_matrix = nx.to_numpy_array(graph)
    # применяем Spectral Clustering
    spectral_clustering = SpectralClustering(n_clusters=num_communities, affinity='nearest_neighbors', random_state=42)
    labels = spectral_clustering.fit_predict(adjacency_matrix)
    # формируем словарь, где ключами являются метки кластеров, а значениями - списки вершин в каждом кластере
    communities = {}
    for node, label in zip(graph.nodes(), labels):
        if label not in communities:
            communities[label] = []
        communities[label].append(node)

    return communities

In [ ]:
# пример использования для графа G_facebook с 5 сообществами
num_communities = 5
facebook_communities = spectral_clustering_community_detection(G_facebook, num_communities)
print("сообщества, выделенные методом Spectral Clustering, для графа G_facebook:")
for label, nodes in facebook_communities.items():
    print(f"сообщество {label + 1}: {nodes}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


сообщества, выделенные методом Spectral Clustering, для графа G_facebook:
сообщество 1: ['0', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '14', '17', '19', '20', '22', '24', '25', '26', '27', '31', '33', '34', '35', '38', '39', '40', '41', '42', '44', '46', '50', '54', '55', '57', '58', '63', '64', '65', '67', '69', '70', '71', '72', '73', '74', '75', '76', '79', '82', '85', '87', '89', '91', '92', '93', '95', '97', '98', '100', '101', '102', '103', '105', '106', '107', '108', '109', '112', '113', '115', '118', '119', '120', '121', '123', '124', '125', '127', '130', '132', '134', '137', '138', '139', '141', '143', '145', '146', '151', '152', '153', '156', '157', '158', '159', '160', '161', '163', '164', '165', '166', '168', '169', '170', '171', '172', '173', '174', '175', '176', '180', '181', '184', '185', '187', '188', '189', '191', '192', '193', '196', '197', '199', '200', '202', '204', '206', '208', '209', '211', '212', '214', '215', '216', '218', '219', '220', '221', '224',

##### 3. Random Walk

In [ ]:
import random

In [ ]:
# выделение сообществ методом Random Walk
def random_walk_community_detection(graph, num_communities, num_iterations=100):
    # выполняем случайные блуждания из случайно выбранных вершин
    random_walks = []
    for _ in range(num_iterations):
        start_node = random.choice(list(graph.nodes()))  # выбираем случайную вершину
        random_walk = [start_node]
        for _ in range(10):  # длина блуждания
            neighbors = list(graph.neighbors(random_walk[-1]))
            if neighbors:
                random_walk.append(random.choice(neighbors))
            else:
                break
        random_walks.append(random_walk)
    # выполняем кластеризацию вершин на основе частоты посещения
    node_visits = {}
    for walk in random_walks:
        for node in walk:
            if node not in node_visits:
                node_visits[node] = 0
            node_visits[node] += 1
    # формируем список вершин, встречающихся хотя бы один раз в блужданиях
    unique_nodes = list(node_visits.keys())
    # инициализируем сообщества
    communities = {i: [] for i in range(num_communities)}
    # проверяем, что каждая вершина графа присутствует в каком-либо сообществе
    num_nodes_graph = len(graph.nodes())
    if len(unique_nodes) < num_nodes_graph:
        # добавляем недостающие вершины в сообщества случайным образом
        missing_nodes = set(graph.nodes()) - set(unique_nodes)
        for node in missing_nodes:
            communities[random.randint(0, num_communities - 1)].append(node)
    # разбиваем уникальные вершины на сообщества
    for i, node in enumerate(unique_nodes):
        communities[i % num_communities].append(node)

    return communities

In [ ]:
# пример использования для графа G_facebook с 5 сообществами
num_communities = 5
facebook_communities = random_walk_community_detection(G_facebook, num_communities)
print("сообщества, выделенные методом Random Walk, для графа G_facebook:")
for label, nodes in facebook_communities.items():
    print(f"сообщество {label + 1}: {nodes}")

сообщества, выделенные методом Random Walk, для графа G_facebook:
сообщество 1: ['1528', '832', '1072', '1786', '3702', '2431', '2071', '1908', '1354', '1941', '2403', '2678', '515', '3007', '1704', '1213', '2887', '2035', '3492', '410', '961', '1157', '1934', '1847', '563', '285', '654', '898', '3352', '66', '2038', '1898', '943', '168', '851', '3071', '3034', '2450', '1478', '2355', '1171', '1392', '2205', '3543', '2043', '470', '864', '1306', '553', '2945', '1438', '3683', '1910', '3028', '145', '1237', '2993', '314', '2117', '2330', '502', '3033', '3919', '775', '1737', '2139', '1944', '2673', '2311', '2525', '3299', '1252', '2538', '2784', '1363', '3440', '357', '345', '880', '692', '1734', '2556', '1841', '3865', '2048', '3915', '4022', '1030', '2367', '3013', '3420', '1809', '3390', '3108', '2721', '535', '3809', '2073', '1330', '1803', '259', '3396', '2995', '764', '3859', '1347', '1563', '552', '3423', '3376', '1640', '3896', '2850', '3665', '441', '8', '1679', '3213', '2660',

##### 4. Node2Vec

In [ ]:
pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.

In [ ]:
from node2vec import Node2Vec

In [ ]:
# выделение сообществ методом Node2Vec
def node2vec_community_detection(graph, num_communities):
    # обучаем модель Node2Vec
    node2vec = Node2Vec(graph, dimensions=128, walk_length=30, num_walks=200, workers=2)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    # получаем векторы вершин
    embeddings = {node: model.wv[node] for node in graph.nodes()}
    # применяем KMeans к векторам вершин для кластеризации
    kmeans = KMeans(n_clusters=num_communities, random_state=42)
    labels = kmeans.fit_predict(list(embeddings.values()))
    # формируем словарь, где ключами являются метки кластеров, а значениями - списки вершин в каждом кластере
    communities = {}
    for node, label in zip(graph.nodes(), labels):
        if label not in communities:
            communities[label] = []
        communities[label].append(node)

    return communities

In [ ]:
# пример использования для графа G_facebook с 5 сообществами
num_communities = 5
facebook_communities = node2vec_community_detection(G_facebook, num_communities)
print("сообщества, выделенные методом Node2Vec, для графа G_facebook:")
for label, nodes in facebook_communities.items():
    print(f"сообщество {label + 1}: {nodes}")

Computing transition probabilities:   0%|          | 0/4039 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


сообщества, выделенные методом Node2Vec, для графа G_facebook:
сообщество 4: ['0', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '14', '17', '19', '20', '22', '24', '25', '26', '27', '31', '33', '34', '35', '38', '39', '40', '41', '42', '44', '46', '50', '54', '55', '57', '63', '64', '65', '67', '69', '70', '71', '72', '73', '74', '75', '76', '79', '82', '85', '87', '89', '91', '92', '93', '95', '97', '98', '100', '101', '102', '103', '105', '106', '108', '109', '112', '113', '115', '118', '119', '120', '121', '123', '124', '125', '127', '130', '132', '134', '137', '138', '139', '141', '143', '145', '146', '151', '152', '153', '156', '157', '158', '159', '160', '161', '163', '164', '165', '166', '168', '169', '170', '171', '172', '173', '174', '175', '176', '180', '181', '184', '185', '187', '188', '189', '191', '192', '193', '196', '197', '199', '200', '202', '204', '206', '208', '209', '211', '212', '214', '215', '216', '218', '219', '220', '221', '224', '225', '226', '230', '2

##### 5. GraphSAGE

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# выделение сообществ методом GraphSAGE
def graphsage_community_detection(graph, num_communities):
    # генерируем соседей вершин для каждой вершины
    neighbors = {node: list(graph.neighbors(node)) for node in graph.nodes()}
    # обучаем модель GraphSAGE на соседях вершин
    model = Word2Vec(neighbors.values(), vector_size=128, window=5, min_count=0, sg=1, workers=2, epochs=1)
    # получаем эмбеддинги вершин
    embeddings = {node: model.wv[node] for node in graph.nodes() if node in model.wv}
    # применяем KMeans к эмбеддингам для кластеризации вершин
    kmeans = KMeans(n_clusters=num_communities, random_state=42)
    labels = kmeans.fit_predict(list(embeddings.values()))
    # формируем словарь, где ключами являются метки кластеров, а значениями - списки вершин в каждом кластере
    communities = {}
    for node, label in zip(embeddings.keys(), labels):
        if label not in communities:
            communities[label] = []
        communities[label].append(node)

    return communities

In [ ]:
# пример использования для графа G_facebook с 5 сообществами
num_communities = 5
facebook_communities = graphsage_community_detection(G_facebook, num_communities)
print("сообщества, выделенные методом GraphSAGE, для графа G_facebook:")
for label, nodes in facebook_communities.items():
    print(f"сообщество {label + 1}: {nodes}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


сообщества, выделенные методом GraphSAGE, для графа G_facebook:
сообщество 2: ['0', '107', '136', '389', '526', '606', '651', '896', '902', '903', '906', '913', '915', '917', '921', '922', '924', '927', '930', '932', '936', '942', '944', '948', '949', '957', '958', '962', '963', '964', '968', '969', '971', '975', '977', '981', '984', '988', '992', '995', '1010', '1011', '1014', '1015', '1019', '1020', '1021', '1032', '1035', '1037', '1042', '1044', '1062', '1066', '1068', '1081', '1082', '1097', '1099', '1100', '1102', '1104', '1106', '1108', '1113', '1114', '1115', '1116', '1118', '1120', '1121', '1122', '1129', '1136', '1139', '1152', '1158', '1161', '1164', '1165', '1166', '1170', '1179', '1180', '1187', '1188', '1190', '1216', '1223', '1226', '1227', '1229', '1231', '1232', '1241', '1245', '1247', '1261', '1265', '1277', '1278', '1279', '1281', '1282', '1283', '1286', '1292', '1296', '1299', '1300', '1304', '1306', '1307', '1308', '1310', '1311', '1314', '1316', '1318', '1320', '13

### Проведение экспериментов

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
# выполнение экспериментов на графе
def run_experiment(graph, num_communities):
    # KMeans
    kmeans_communities = kmeans_community_detection(graph, num_communities)
    kmeans_performance = evaluate_communities(graph, kmeans_communities)
    # Spectral Clustering
    spectral_communities = spectral_clustering_community_detection(graph, num_communities)
    spectral_performance = evaluate_communities(graph, spectral_communities)
    # Random Walk
    random_walk_communities = random_walk_community_detection(graph, num_communities)
    random_walk_performance = evaluate_communities(graph, random_walk_communities)
    # Node2Vec
    node2vec_communities = node2vec_community_detection(graph, num_communities)
    node2vec_performance = evaluate_communities(graph, node2vec_communities)
    # GraphSAGE
    graphsage_communities = graphsage_community_detection(graph, num_communities)
    graphsage_performance = evaluate_communities(graph, graphsage_communities)
    return {
        "KMeans": kmeans_performance,
        "Spectral Clustering": spectral_performance,
        "Random Walk": random_walk_performance,
        "Node2Vec": node2vec_performance,
        "GraphSAGE": graphsage_performance
    }

In [ ]:
# оценка эффективности и интерпретируемости сообществ
def evaluate_communities(graph, communities):
    # получаем общее количество вершин в графе
    num_nodes_graph = len(graph.nodes())
    # собираем все узлы в сообщества
    all_nodes = []
    all_labels = []
    for label, nodes in communities.items():
        all_nodes.extend(nodes)
        all_labels.extend([label] * len(nodes))
    # заполняем отсутствующие узлы меткой по умолчанию
    missing_nodes = set(graph.nodes()) - set(all_nodes)
    default_label = len(communities)
    all_labels.extend([default_label] * len(missing_nodes))
    # обеспечиваем согласованность между метками и узлами
    assert len(all_labels) == num_nodes_graph, "кол-во узлов не = кол-во меток"
    # вычисляем индекс силуэта
    silhouette = silhouette_score(nx.to_numpy_array(graph), all_labels)
    # оцениваем интерпретируемость
    # средний размер сообщества
    average_community_size = sum(len(community) for community in communities.values()) / len(communities)

    return {
        "Silhouette": silhouette,
        "Interpretability": average_community_size
    }

In [ ]:
# вывод результатов эксперимента
def print_results(results):
    for method, performance in results.items():
        print(f"Method: {method}")
        print(f"Silhouette Score: {performance['Silhouette']}")
        print(f"Average Community Size: {performance['Interpretability']}")
        print()

In [ ]:
# подготовка данных
num_communities = 5

# запуск экспериментов для графа G_facebook
facebook_results = run_experiment(G_facebook, num_communities)

# вывод результатов экспериментов
print("Results for Facebook dataset:")
print_results(facebook_results)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Computing transition probabilities:   0%|          | 0/4039 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Results for Facebook dataset:
Method: KMeans
Silhouette Score: -0.09883511856566277
Average Community Size: 807.8

Method: Spectral Clustering
Silhouette Score: -0.1504888468368776
Average Community Size: 807.8

Method: Random Walk
Silhouette Score: -0.06592160680111325
Average Community Size: 807.8

Method: Node2Vec
Silhouette Score: -0.07574242667499384
Average Community Size: 807.8

Method: GraphSAGE
Silhouette Score: -0.06132640387102475
Average Community Size: 807.8



In [ ]:
# запуск экспериментов для графа G_googleplus
googleplus_results = run_experiment(G_googleplus, num_communities)

# вывод результатов экспериментов
print("Results for Google Plus dataset:")
print_results(googleplus_results)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Computing transition probabilities:   0%|          | 0/7805 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Results for Google Plus dataset:
Method: KMeans
Silhouette Score: -0.20767496250477707
Average Community Size: 1561.0

Method: Spectral Clustering
Silhouette Score: -0.3265362907820944
Average Community Size: 1561.0

Method: Random Walk
Silhouette Score: -0.11460348703954983
Average Community Size: 1561.0

Method: Node2Vec
Silhouette Score: -0.028901735140876572
Average Community Size: 1561.0

Method: GraphSAGE
Silhouette Score: -0.23439433655236386
Average Community Size: 1553.6



In [ ]:
# запуск экспериментов для графа G_twitter
twitter_results = run_experiment(G_twitter, num_communities)

# вывод результатов экспериментов
print("Results for Twitter dataset:")
print_results(twitter_results)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_spectral.py:717: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(


Computing transition probabilities:   0%|          | 0/2455 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Results for Twitter dataset:
Method: KMeans
Silhouette Score: -0.10661740506863611
Average Community Size: 491.0

Method: Spectral Clustering
Silhouette Score: -0.2732650276380718
Average Community Size: 491.0

Method: Random Walk
Silhouette Score: -0.049165877252855865
Average Community Size: 491.0

Method: Node2Vec
Silhouette Score: -0.08803986470335184
Average Community Size: 491.0

Method: GraphSAGE
Silhouette Score: -0.24776739812334989
Average Community Size: 480.6



### Анализ и оценка результатов экспериментов

Из результатов экспериментов видно, что каждый метод выделения сообществ имеет свои преимущества и недостатки:

1) **KMeans**:
   - **Facebook dataset**: Silhouette Score: -0.0988, Average Community Size: 807.8
   - **Google Plus dataset**: Silhouette Score: -0.2077, Average Community Size: 1561.0
   - **Twitter dataset**: Silhouette Score: -0.1066, Average Community Size: 491.0
   - **Выводы**: KMeans показывает средние результаты по силуэтному коэффициенту. Размеры сообществ сильно различаются в зависимости от датасета.

2) **Spectral Clustering**:
   - **Facebook dataset**: Silhouette Score: -0.1505, Average Community Size: 807.8
   - **Google Plus dataset**: Silhouette Score: -0.3265, Average Community Size: 1561.0
   - **Twitter dataset**: Silhouette Score: -0.2733, Average Community Size: 491.0
   - **Выводы**: Spectral Clustering показывает лучшие результаты по силуэтному коэффициенту, но также имеет большие размеры сообществ.

3) **Random Walk**:
   - **Facebook dataset**: Silhouette Score: -0.0659, Average Community Size: 807.8
   - **Google Plus dataset**: Silhouette Score: -0.1146, Average Community Size: 1561.0
   - **Twitter dataset**: Silhouette Score: -0.0492, Average Community Size: 491.0
   - **Выводы**: Random Walk обеспечивает средние результаты по силуэтному коэффициенту и схожие размеры сообществ.

4) **Node2Vec**:
   - **Facebook dataset**: Silhouette Score: -0.0757, Average Community Size: 807.8
   - **Google Plus dataset**: Silhouette Score: -0.0289, Average Community Size: 1561.0
   - **Twitter dataset**: Silhouette Score: -0.0880, Average Community Size: 491.0
   - **Выводы**: Node2Vec имеет средние значения силуэтного коэффициента, но размеры сообществ более уравновешены.

5) **GraphSAGE**:
   - **Facebook dataset**: Silhouette Score: -0.0613, Average Community Size: 807.8
   - **Google Plus dataset**: Silhouette Score: -0.2344, Average Community Size: 1553.6
   - **Twitter dataset**: Silhouette Score: -0.2478, Average Community Size: 480.6
   - **Выводы**: GraphSAGE имеет средние результаты по силуэтному коэффициенту, но немного более уравновешенные сообщества.

**Выводы**:

1. Спектральная кластеризация (Spectral Clustering) демонстрирует лучшие результаты по силуэтному коэффициенту на всех датасетах, но при этом формирует сообщества с более большими размерами. Она подходит для случаев, когда интерпретация кластеров не так важна, как их качество.
   
2. Node2Vec и GraphSAGE показывают хорошие результаты по силуэтному коэффициенту и формируют более уравновешенные сообщества по сравнению с другими методами. Они могут быть предпочтительными, когда важны как качество, так и интерпретируемость кластеров.

3. KMeans и Random Walk обеспечивают средние результаты по силуэтному коэффициенту и формируют сообщества с различными размерами. Они могут быть полезны в случаях, когда нужно быстро получить первичные результаты без слишком больших вычислительных затрат.

**Рекомендации**:

- Для задач, где важно получить высокое качество кластеризации, но не так важна интерпретируемость кластеров, следует использовать спектральную кластеризацию (Spectral Clustering).
  
- Если необходимо сбалансировать между качеством и интерпретируемостью кластеров, рекомендуется использовать Node2Vec или GraphSAGE.
  
- Для быстрой оценки кластеров без больших вычислительных затрат можно применять KMeans или Random Walk.